In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OneHotEncoder
from torch.utils.data import TensorDataset, DataLoader

In [161]:
data = pd.read_csv('news_data.csv')

In [162]:
stock_tickers = data['Symbol'].unique()

In [163]:
stock_df = data[data['Symbol'] == 'AAPL'].copy()

drop_cols = ['Security', 'GICS Sub-Industry', 'Symbol', 'Adj Close']

stock_df.drop(columns=drop_cols, axis=1, inplace=True)
#aapl.drop('GICS Sub-Industry', axis=1, inplace=True)
#aapl.drop('Symbol', axis=1, inplace=True)
stock_df = pd.get_dummies(stock_df, columns=['GICS Sector'])
stock_df


,Date,Open,High,Low,Close,Volume,News - All News Volume,News - Volume,News - Positive Sentiment,News - Negative Sentiment,...,News - Stocks,News - Dividends,News - Corporate Earnings,News - Mergers & Acquisitions,News - Store Openings,News - Product Recalls,News - Adverse Events,News - Personnel Changes,News - Stock Rumors,GICS Sector_Information Technology
19845,2020-09-30,113.790001,117.260002,113.620003,115.809998,142675200.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
19846,2020-10-01,117.639999,117.720001,115.830002,116.790001,116120400.0,589369.0,1178.0,120.0,72.0,...,10.0,0.0,3.0,4.0,0.0,0.0,19.0,0.0,0.0,1
19847,2020-10-02,112.889999,115.370003,112.220001,113.019997,144712000.0,506082.0,1285.0,145.0,62.0,...,16.0,0.0,11.0,5.0,9.0,0.0,14.0,0.0,0.0,1
19848,2020-10-05,113.910004,116.650002,113.550003,116.500000,106243800.0,489872.0,1315.0,199.0,71.0,...,27.0,1.0,9.0,2.0,0.0,10.0,99.0,1.0,0.0,1
19849,2020-10-06,115.699997,116.120003,112.250000,113.160004,161498200.0,550713.0,2287.0,256.0,79.0,...,9.0,1.0,5.0,11.0,13.0,4.0,51.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20281,2022-06-24,139.899994,141.910004,139.770004,141.660004,89116800.0,533627.0,1487.0,110.0,75.0,...,29.0,0.0,1.0,13.0,0.0,0.0,129.0,0.0,0.0,1
20282,2022-06-27,142.699997,143.490005,140.970001,141.660004,70207900.0,433341.0,1375.0,156.0,44.0,...,31.0,0.0,8.0,2.0,0.0,0.0,26.0,0.0,0.0,1
20283,2022-06-28,142.130005,143.419998,137.320007,137.440002,67083400.0,559643.0,1288.0,106.0,46.0,...,34.0,0.0,6.0,5.0,0.0,0.0,33.0,15.0,0.0,1
20284,2022-06-29,137.460007,140.669998,136.669998,139.229996,66242400.0,536364.0,1580.0,106.0,78.0,...,40.0,0.0,4.0,6.0,0.0,0.0,25.0,1.0,0.0,1


In [164]:
stock_df['Date'] = pd.to_datetime(stock_df['Date'], format='%Y/%m/%d')
stock_df.sort_values('Date', inplace=True)

In [165]:
stock_df['Year'] = stock_df['Date'].dt.year
stock_df['Month'] = stock_df['Date'].dt.month
stock_df['Day'] = stock_df['Date'].dt.day


# You can decide to drop the original 'week_date' if you don't need it anymore
stock_df.drop('Date', axis=1, inplace=True)

In [166]:
stock_df

,Open,High,Low,Close,Volume,News - All News Volume,News - Volume,News - Positive Sentiment,News - Negative Sentiment,News - New Products,...,News - Mergers & Acquisitions,News - Store Openings,News - Product Recalls,News - Adverse Events,News - Personnel Changes,News - Stock Rumors,GICS Sector_Information Technology,Year,Month,Day
19845,113.790001,117.260002,113.620003,115.809998,142675200.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,2020,9,30
19846,117.639999,117.720001,115.830002,116.790001,116120400.0,589369.0,1178.0,120.0,72.0,111.0,...,4.0,0.0,0.0,19.0,0.0,0.0,1,2020,10,1
19847,112.889999,115.370003,112.220001,113.019997,144712000.0,506082.0,1285.0,145.0,62.0,71.0,...,5.0,9.0,0.0,14.0,0.0,0.0,1,2020,10,2
19848,113.910004,116.650002,113.550003,116.500000,106243800.0,489872.0,1315.0,199.0,71.0,43.0,...,2.0,0.0,10.0,99.0,1.0,0.0,1,2020,10,5
19849,115.699997,116.120003,112.250000,113.160004,161498200.0,550713.0,2287.0,256.0,79.0,204.0,...,11.0,13.0,4.0,51.0,0.0,0.0,1,2020,10,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20281,139.899994,141.910004,139.770004,141.660004,89116800.0,533627.0,1487.0,110.0,75.0,31.0,...,13.0,0.0,0.0,129.0,0.0,0.0,1,2022,6,24
20282,142.699997,143.490005,140.970001,141.660004,70207900.0,433341.0,1375.0,156.0,44.0,136.0,...,2.0,0.0,0.0,26.0,0.0,0.0,1,2022,6,27
20283,142.130005,143.419998,137.320007,137.440002,67083400.0,559643.0,1288.0,106.0,46.0,67.0,...,5.0,0.0,0.0,33.0,15.0,0.0,1,2022,6,28
20284,137.460007,140.669998,136.669998,139.229996,66242400.0,536364.0,1580.0,106.0,78.0,62.0,...,6.0,0.0,0.0,25.0,1.0,0.0,1,2022,6,29


In [167]:

for i in range(1, 11):
    stock_df[f'High_lag_{i}'] = stock_df['High'].shift(i)
    stock_df[f'Low_lag_{i}'] = stock_df['Low'].shift(i)
    stock_df[f'Volume_lag_{i}'] = stock_df['Volume'].shift(i)
    stock_df[f'Open_lag_{i}'] = stock_df['Open'].shift(i)
    stock_df[f'Close_lag_{i}'] = stock_df['Close'].shift(i)

open_pred_df = stock_df.copy()
close_pred_df = stock_df.copy()



# Drop rows with NaN values (first 20 rows)
close_pred_df = close_pred_df.dropna().reset_index(drop=True)
open_pred_df = open_pred_df.dropna().reset_index(drop=True)
stock_df = stock_df.dropna().reset_index(drop=True)
 



In [168]:
open_pred_df

,Open,High,Low,Close,Volume,News - All News Volume,News - Volume,News - Positive Sentiment,News - Negative Sentiment,News - New Products,...,High_lag_9,Low_lag_9,Volume_lag_9,Open_lag_9,Close_lag_9,High_lag_10,Low_lag_10,Volume_lag_10,Open_lag_10,Close_lag_10
0,121.000000,123.029999,119.620003,121.190002,150712000.0,577765.0,3985.0,624.0,142.0,516.0,...,117.720001,115.830002,116120400.0,117.639999,116.790001,117.260002,113.620003,142675200.0,113.790001,115.809998
1,118.720001,121.199997,118.150002,120.709999,112559200.0,580934.0,2035.0,268.0,95.0,186.0,...,115.370003,112.220001,144712000.0,112.889999,113.019997,117.720001,115.830002,116120400.0,117.639999,116.790001
2,121.279999,121.550003,118.809998,119.019997,115393800.0,530759.0,2214.0,261.0,121.0,198.0,...,116.650002,113.550003,106243800.0,113.910004,116.500000,115.370003,112.220001,144712000.0,112.889999,113.019997
3,119.959999,120.419998,115.660004,115.980003,120639300.0,482043.0,1625.0,177.0,79.0,60.0,...,116.120003,112.250000,161498200.0,115.699997,113.160004,116.650002,113.550003,106243800.0,113.910004,116.500000
4,116.199997,118.980003,115.629997,117.510002,124423700.0,550949.0,2117.0,269.0,93.0,135.0,...,115.550003,114.129997,96849000.0,114.620003,115.080002,116.120003,112.250000,161498200.0,115.699997,113.160004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426,139.899994,141.910004,139.770004,141.660004,89116800.0,533627.0,1487.0,110.0,75.0,31.0,...,140.759995,137.059998,91437900.0,140.279999,137.130005,147.949997,142.529999,69473000.0,147.080002,142.639999
427,142.699997,143.490005,140.970001,141.660004,70207900.0,433341.0,1375.0,156.0,44.0,136.0,...,135.199997,131.440002,122207100.0,132.869995,131.880005,140.759995,137.059998,91437900.0,140.279999,137.130005
428,142.130005,143.419998,137.320007,137.440002,67083400.0,559643.0,1288.0,106.0,46.0,67.0,...,133.889999,131.479996,84784300.0,133.130005,132.759995,135.199997,131.440002,122207100.0,132.869995,131.880005
429,137.460007,140.669998,136.669998,139.229996,66242400.0,536364.0,1580.0,106.0,78.0,62.0,...,137.339996,132.160004,91533000.0,134.289993,135.429993,133.889999,131.479996,84784300.0,133.130005,132.759995


In [169]:
y_open = stock_df['Open'].values 
y_close = stock_df['Close'].values

open_pred_df.drop('Open', axis=1, inplace=True)
open_pred_df.drop('Close', axis=1, inplace=True)
close_pred_df.drop('Close', axis=1, inplace=True)

In [170]:
close_pred_df

,Open,High,Low,Volume,News - All News Volume,News - Volume,News - Positive Sentiment,News - Negative Sentiment,News - New Products,News - Layoffs,...,High_lag_9,Low_lag_9,Volume_lag_9,Open_lag_9,Close_lag_9,High_lag_10,Low_lag_10,Volume_lag_10,Open_lag_10,Close_lag_10
0,121.000000,123.029999,119.620003,150712000.0,577765.0,3985.0,624.0,142.0,516.0,0.0,...,117.720001,115.830002,116120400.0,117.639999,116.790001,117.260002,113.620003,142675200.0,113.790001,115.809998
1,118.720001,121.199997,118.150002,112559200.0,580934.0,2035.0,268.0,95.0,186.0,0.0,...,115.370003,112.220001,144712000.0,112.889999,113.019997,117.720001,115.830002,116120400.0,117.639999,116.790001
2,121.279999,121.550003,118.809998,115393800.0,530759.0,2214.0,261.0,121.0,198.0,0.0,...,116.650002,113.550003,106243800.0,113.910004,116.500000,115.370003,112.220001,144712000.0,112.889999,113.019997
3,119.959999,120.419998,115.660004,120639300.0,482043.0,1625.0,177.0,79.0,60.0,0.0,...,116.120003,112.250000,161498200.0,115.699997,113.160004,116.650002,113.550003,106243800.0,113.910004,116.500000
4,116.199997,118.980003,115.629997,124423700.0,550949.0,2117.0,269.0,93.0,135.0,0.0,...,115.550003,114.129997,96849000.0,114.620003,115.080002,116.120003,112.250000,161498200.0,115.699997,113.160004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426,139.899994,141.910004,139.770004,89116800.0,533627.0,1487.0,110.0,75.0,31.0,0.0,...,140.759995,137.059998,91437900.0,140.279999,137.130005,147.949997,142.529999,69473000.0,147.080002,142.639999
427,142.699997,143.490005,140.970001,70207900.0,433341.0,1375.0,156.0,44.0,136.0,2.0,...,135.199997,131.440002,122207100.0,132.869995,131.880005,140.759995,137.059998,91437900.0,140.279999,137.130005
428,142.130005,143.419998,137.320007,67083400.0,559643.0,1288.0,106.0,46.0,67.0,1.0,...,133.889999,131.479996,84784300.0,133.130005,132.759995,135.199997,131.440002,122207100.0,132.869995,131.880005
429,137.460007,140.669998,136.669998,66242400.0,536364.0,1580.0,106.0,78.0,62.0,4.0,...,137.339996,132.160004,91533000.0,134.289993,135.429993,133.889999,131.479996,84784300.0,133.130005,132.759995


In [171]:
#numerical_cols_open = open_pred_df.select_dtypes(include=['number']).columns
#numerical_cols_close = close_pred_df.select_dtypes(include=['number']).columns


scaler = StandardScaler()

open_pred_df = scaler.fit_transform(open_pred_df)
close_pred_df = scaler.fit_transform(close_pred_df)


#open_pred_df[numerical_cols_open] = scaler.fit_transform(open_pred_df[numerical_cols])
#close_pred_df[numerical_cols_close] = scaler.fit_transform(close_pred_df[numerical_cols])


In [172]:
# DATA FOR PREDICTING OPEN PRICE 

X_open = open_pred_df.copy()

# Split the data into training and remaining data (validation + test)
X_train_open, X_remaining_open, y_train_open, y_remaining_open = train_test_split(X_open, y_open, test_size=0.3, random_state=42)

# Split the remaining data into validation and test sets
X_val_open, X_test_open, y_val_open, y_test_open = train_test_split(X_remaining_open, y_remaining_open, test_size=0.5, random_state=42)



# DATA FOR PREDICTING CLOSE PRICE

X_close = close_pred_df.copy()

# Split the data into training and remaining data (validation + test)
X_train_close, X_remaining_close, y_train_close, y_remaining_close = train_test_split(X_close, y_close, test_size=0.3, random_state=42)

# Split the remaining data into validation and test sets
X_val_close, X_test_close, y_val_close, y_test_close = train_test_split(X_remaining_open, y_remaining_open, test_size=0.5, random_state=42)



In [ ]:
# TURN INTO TENSORS / TORCH TENSORS 

